In [11]:
from sklearn import datasets 
import matplotlib.pyplot as plt 
import seaborn as sns
import math
import numpy as np
from scipy import stats
from tqdm import tqdm

data = datasets.make_classification(100,20,n_classes = 2)


In [18]:
data[1]

array([0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1])

In [19]:
X,Y = data[0],data[1]

In [20]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

model = AdaBoostClassifier()
model.fit(X,Y)
print(accuracy_score(Y,model.predict(X)))


1.0


# Implementation

In [78]:
class Stump:
    
    def __init__(self):
        
        self.threshold = None 
        self.feature = None
        
        # if the threshold >= unique_features_vals then polarity +1 else -1
        #this polarity acts as the left and right leafs 
        self.polarity = None 
        
        #gives the importance 
        self.alpha  = None 
        
        

class AdaStump:
    
    
    def __init__(self,n_stumps = 5):
        
        self.n_stumps = n_stumps
        self.stumps = []

        for i in range(self.n_stumps):
            self.stumps.append(Stump())
    
    
    def process_label(self,y):
        
        if 1 in np.unique(y) and -1 in np.unique(y):
            return y
        else:
            y[y == 0] = -1
            
            return y
            
            
    def fit(self,X,y):
        
        
        Y =  self.process_label(y)
        
        print(np.unique(y))

        n_samples,n_features = X.shape
        
        w = np.full(y.shape,(1 / n_samples))
        
        #get the best unique value from the best fearure_i that gives maximum error
        
        for i in range(self.n_stumps):
            
            min_error = float('inf')
        
            for feature_i in range(n_features):

    
                #go throght the unique values of feature_i
                for unique_val in np.unique(X[:,feature_i]):

                    # if feature_i >= unique_val then polarity is +1 else polarity is -1

                    #error ? miscalssify ? if polarity != y_true
                    #so each stump can only be used for binary classification 
                    p = 1
                    
                    polarity = np.ones(y.shape)

                    polarity[X[:,feature_i] < unique_val] = -1


                    #weights to calculate to error why? 
                    #it acts as a metric to minimize ,assign more weightage 

                    error = sum(w[polarity != y])
#                     print(error)

                    if error > 0.5:
                        error = 1 - error
                        p = -1

                        
                    if error < min_error:
                        
                        self.stumps[i].polarity = p
                        self.stumps[i].threshold = unique_val
                        self.stumps[i].feature = feature_i
                        min_error = error  
                       
                    
            self.stumps[i].alpha = 0.5 * math.log((1.0 - min_error) / (min_error + 1e-10))        


            predictions = np.ones(np.shape(y))


    #         print(self.stumps[i].polarity,self.stumps[i].feature,self.stumps[i].threshold)    

            negative_idx = (self.stumps[i].polarity * X[:, self.stumps[i].feature] < self.stumps[i].polarity * self.stumps[i].threshold)

            predictions[negative_idx] = -1

            w *= np.exp(-self.stumps[i].alpha * y * predictions)

            w /= np.sum(w)
        
        
    def predicts(self,X):
        
      
        
        y_pred = np.zeros((X.shape[0]))
        
        for i in range(self.n_stumps):
            
            predictions = np.ones(np.shape(y_pred))
            # The indexes where the sample values are below threshold
            
            
            
            negative_idx = (self.stumps[i].polarity * X[:, self.stumps[i].feature] < self.stumps[i].polarity * self.stumps[i].threshold)
            # Label those as '-1'
            predictions[negative_idx] = -1
        
            y_pred += self.stumps[i].alpha * predictions 
            
        
        y_pred = np.sign(y_pred).flatten()
        
        return y_pred
            
        
model  = AdaStump()       
model.fit(X,Y)
accuracy_score(Y,model.predicts(X))

[-1  1]


0.94

In [52]:
# 0 in np.unique([1,1,1,1,0,0,0]) and 1 in np.unique([1,1,1,1,0,0,0])
min_error = float('-inf')
min_error 

-inf